In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import time
#import winsound

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss, brier_score_loss
from sklearn.ensemble import RandomForestClassifier
import lightgbm
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

In [2]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
# df_original = pd.read_csv("diabetes dataset.csv")


MODEL = "lgbm"
#MODEL = "xgb"
sound = False                            # beep after each validation, beep extra after all validations
include_original = False                 # use the original database as additional training data?
validate_models = False                  # compare a lot of models with different model parameters?
submit_prediction = not validate_models # create a submission?
verbose = False                         # print a lot of variables for debugging purposes?

In [3]:
# df_original

In [4]:
df_train

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty,exam_score
0,0,21,female,b.sc,7.91,98.8,no,4.9,average,online videos,low,easy,78.300
1,1,18,other,diploma,4.95,94.8,yes,4.7,poor,self-study,medium,moderate,46.700
2,2,20,female,b.sc,4.68,92.6,yes,5.8,poor,coaching,high,moderate,99.000
3,3,19,male,b.sc,2.00,49.5,yes,8.3,average,group study,high,moderate,63.900
4,4,23,male,bca,7.65,86.9,yes,9.6,good,self-study,high,easy,100.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
629995,629995,18,female,b.tech,4.86,70.7,yes,4.1,good,mixed,high,moderate,69.500
629996,629996,21,female,ba,7.08,54.4,yes,4.5,average,mixed,low,moderate,78.900
629997,629997,24,male,bca,0.64,44.2,yes,4.3,poor,online videos,low,moderate,19.599
629998,629998,20,male,b.com,1.54,75.1,yes,8.2,average,group study,high,moderate,59.100


In [5]:
df_test
# df_train['id'].dtype

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty
0,630000,24,other,ba,6.85,65.2,yes,5.2,poor,group study,high,easy
1,630001,18,male,diploma,6.61,45.0,no,9.3,poor,coaching,low,easy
2,630002,24,female,b.tech,6.60,98.5,yes,6.2,good,group study,medium,moderate
3,630003,24,male,diploma,3.03,66.3,yes,5.7,average,mixed,medium,moderate
4,630004,20,female,b.tech,2.03,42.4,yes,9.2,average,coaching,low,moderate
...,...,...,...,...,...,...,...,...,...,...,...,...
269995,899995,21,other,b.com,2.55,82.3,yes,8.4,average,mixed,medium,hard
269996,899996,17,female,b.com,0.49,46.4,yes,8.8,good,mixed,low,easy
269997,899997,22,male,bba,6.62,74.7,yes,5.5,good,coaching,high,easy
269998,899998,22,other,ba,4.08,51.8,yes,8.7,poor,online videos,high,moderate


In [6]:
import os
import sys


def suppress_python_output():
    """
    Redirect Python-level stdout/stderr to os.devnull.
    Idempotent: repeated calls do nothing.
    """
    if getattr(suppress_python_output, "_active", False):
        return
    suppress_python_output._old_stdout = sys.stdout
    suppress_python_output._old_stderr = sys.stderr
    suppress_python_output._devnull = open(os.devnull, "w")
    sys.stdout = suppress_python_output._devnull
    sys.stderr = suppress_python_output._devnull
    suppress_python_output._active = True


def restore_python_output():
    """
    Restore Python-level stdout/stderr previously redirected by suppress_python_output.
    Safe to call even if not active.
    """
    if not getattr(suppress_python_output, "_active", False):
        return
    try:
        sys.stdout.flush()
    except Exception:
        pass
    try:
        sys.stderr.flush()
    except Exception:
        pass
    # close devnull and restore originals
    try:
        suppress_python_output._devnull.close()
    except Exception:
        pass
    sys.stdout = suppress_python_output._old_stdout
    sys.stderr = suppress_python_output._old_stderr
    suppress_python_output._active = False

In [7]:
# concat df_train and df_original to df_train
# ignore columns that exist in df_original only
# fill the id column for rows from df_original with values from -100000 to -1
originals = 1
for _ in range(originals):
    if include_original:
        df_orig_aligned = df_original.reindex(columns=df_train.columns)  # drop columns that exist only in df_original
        start = -len(df_original)
        ids = np.arange(start, 0)
        try:  # try to cast ids to the same dtype as df_train['id'], fallback to object
            ids = ids.astype(df_train['id'].dtype)
        except Exception:
            ids = ids.astype(object)

        df_orig_aligned.loc[:, 'id'] = ids
        df_train = pd.concat([df_train, df_orig_aligned], ignore_index=True, sort=False)  # concat keeping df_train's column set
df_train

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty,exam_score
0,0,21,female,b.sc,7.91,98.8,no,4.9,average,online videos,low,easy,78.300
1,1,18,other,diploma,4.95,94.8,yes,4.7,poor,self-study,medium,moderate,46.700
2,2,20,female,b.sc,4.68,92.6,yes,5.8,poor,coaching,high,moderate,99.000
3,3,19,male,b.sc,2.00,49.5,yes,8.3,average,group study,high,moderate,63.900
4,4,23,male,bca,7.65,86.9,yes,9.6,good,self-study,high,easy,100.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
629995,629995,18,female,b.tech,4.86,70.7,yes,4.1,good,mixed,high,moderate,69.500
629996,629996,21,female,ba,7.08,54.4,yes,4.5,average,mixed,low,moderate,78.900
629997,629997,24,male,bca,0.64,44.2,yes,4.3,poor,online videos,low,moderate,19.599
629998,629998,20,male,b.com,1.54,75.1,yes,8.2,average,group study,high,moderate,59.100


In [8]:
# find missing values
missing_values = df_train.isnull().sum()
missing_values = missing_values[missing_values > 0]

if not missing_values.empty:
    plt.figure(figsize=(10, 6))
    sns.barplot(x=missing_values.index, y=missing_values.values, palette='viridis')
    plt.xticks(rotation=90)
    plt.xlabel('Features')
    plt.ylabel('Missing Values')
    plt.title('Missing Values per Feature')
    plt.tight_layout()
    plt.show()
else:
    print("✅ No missing values found in the dataset.")

✅ No missing values found in the dataset.


In [9]:
# replace all inf/-inf with NaN
df_test.replace([np.inf, -np.inf], 0, inplace=True)
df_test.fillna(0, inplace=True)

df_train.replace([np.inf, -np.inf], 0, inplace=True)
df_train.fillna(0, inplace=True)

In [10]:
# print all existing categories and their percentages.
if verbose:
    cols = ["gender", "course", "internet_access", "sleep_quality", "study_method", "facility_rating"]
    for col in cols:
        print(f"\nColumn: `{col}`")
        vc = df_train[col].value_counts(dropna=False, normalize=True) * 100
        for name, pct in vc.items():
            print(f"  {name:13}: {pct:6.2f}%")

In [11]:
# map categories, e.g. male->0, female->1
def map_column_values(col, mapping, dataframes):
    # Case-insensitive mapping that preserves missing values
    map_lower = {k.lower(): v for k, v in mapping.items()}

    def map_series_ci(series, mapping_lower):
        mask_na = series.isna()
        s = series.astype(str).str.strip().str.lower()
        s = s.where(~mask_na, pd.NA)  # keep original missing as <NA>
        return s.map(mapping_lower).astype("Int64")

    for df in dataframes:
        df[col] = map_series_ci(df[col], map_lower)
        values = df[col].value_counts(dropna=False)
        if verbose:
            print(values)

In [12]:
# mapping = {"Female": 0, "Other": 1,"Male": 2}
# map_column_values("gender", mapping, [df_train, df_test])
#
# mapping = {"No formal": 0, "Highschool": 1, "Graduate": 2, "Postgraduate": 3}
# map_column_values("education_level", mapping, [df_train, df_test])
#
# mapping = {"Low": 0, "Lower-Middle": 1, "Middle": 2, "Upper-Middle": 3, "High": 4}
# map_column_values("income_level", mapping, [df_train, df_test])
#
# mapping = {"Unemployed": 0, "Retired": 1, "Employed": 2, "Student": 3}
# map_column_values("employment_status", mapping, [df_train, df_test])
#
# mapping = {"Never": 0, "Former": 1, "Current": 2}
# map_column_values("smoking_status", mapping, [df_train, df_test])

In [13]:
def add_derived_features(dataframes):
    for df in dataframes:
        # tri_term = df["triglycerides"] / 88.5 / np.where(gender_female, 0.81, 1.03)


        #df["physical_inactivity"] = 1.0 / (df["physical_activity_minutes_per_week"] + 0.0000001)


        #df.drop(columns=["age"], inplace=True)


        pass

add_derived_features([df_train, df_test])
one_hot_features = ["course", "gender","internet_access", "sleep_quality", "study_method", "facility_rating", "exam_difficulty"]
# one_hot_features = ["ethnicity", "gender", "education_level", "income_level", "employment_status", "smoking_status"]

In [14]:
# df_test

In [15]:
# one hot encoding
def one_hot_encode(df, features):
    df_out = df.copy()
    dummies = pd.get_dummies(df_out[features], prefix=features, dummy_na=False)  # one hot encode
    df_out = df_out.drop(columns=features)  # remove original features from df
    df_out = pd.concat([df_out, dummies], axis=1)  # add one hot encoded columns to df
    return df_out

X_training = one_hot_encode(df_train, one_hot_features) # One hot encode training set
X_test = one_hot_encode(df_test, one_hot_features) # One hot encode test set
# X_training

In [16]:
# X_test
# 57, 0, 400, 5.9, 6.8, 11, 22, 0.86, 132, 85, 62 (heart rate)

In [17]:
# the target
y_init = X_training["exam_score"]


# transform y into an array
y_str = y_init.astype(str)
le = LabelEncoder().fit(y_str)
y = le.transform(y_str)
# X_training

In [18]:
y_init

0          78.300
1          46.700
2          99.000
3          63.900
4         100.000
           ...   
629995     69.500
629996     78.900
629997     19.599
629998     59.100
629999     37.200
Name: exam_score, Length: 630000, dtype: float64

In [19]:
# how many different classes does the target have?
n_classes = len(np.unique(y))
if verbose:
    print("Target shape:", getattr(y, "shape", None), "unique classes:", np.unique(y))
    print(f"{n_classes=}")

In [20]:
def generate_model_kwargs(n_classes, n_estimators=2000, learning_rate=0.03, max_depth=4, subsample=0.8, colsample_bytree=0.8, early_stopping_rounds=50):
    if n_classes == 0:
        raise RuntimeError("Target `y` is empty — check earlier processing (dropped column or filled with NaN).")
    if n_classes == 1:
        raise RuntimeError(f"Only one class present ({np.unique(y)[0]}). Cannot train a classifier.")
    if n_classes == 2:
        model_kwargs = {
            "objective": "binary:logistic",  # choose objective and eval_metric based on number of classes
            "eval_metric": "logloss",
            "n_estimators": n_estimators,
            "learning_rate": learning_rate,
            "max_depth": max_depth,
            "subsample": subsample,
            "colsample_bytree": colsample_bytree,
            "tree_method": "hist",
            "early_stopping_rounds": early_stopping_rounds,
        }
    elif n_classes > 2:
        model_kwargs = {
            "objective": "multi:softprob",
            "num_class": n_classes,
            "eval_metric": "mlogloss",
            "n_estimators": n_estimators,
            "learning_rate": learning_rate,
            "max_depth": max_depth,
            "subsample": subsample,
            "colsample_bytree": colsample_bytree,
            "tree_method": "hist",
            "early_stopping_rounds": early_stopping_rounds,
        }
    else:
        print("Error")
    return model_kwargs

In [21]:
# # COMPARING MODELS PATH
def validate_model(n_estimators, learning_rate, max_depth, subsample, colsample_bytree, early_stopping_rounds, model="xgb"):
    # kwargs = generate_model_kwargs(n_classes, n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth,
    #                                      subsample=subsample, colsample_bytree=colsample_bytree, early_stopping_rounds=early_stopping_rounds)
    n_splits = 5
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    if n_classes == 2:
        oof_preds = np.zeros(len(y), dtype=float)
    else:
        oof_preds = np.zeros((len(y), n_classes), dtype=float)

    for fold, (tr_idx, val_idx) in enumerate(skf.split(X_training, y), start=1):
        X_tr, X_val = X_training.iloc[tr_idx], X_training.iloc[val_idx]
        y_tr, y_val = y[tr_idx], y[val_idx]

        if model == "xgb":  # XGBoost
            if n_classes == 2:
                kwargs = {
                    "objective": "binary:logistic",  # choose objective and eval_metric based on number of classes
                    "eval_metric": "logloss",
                    "n_estimators": n_estimators,
                    "learning_rate": learning_rate,
                    "max_depth": max_depth,
                    "subsample": subsample,
                    "colsample_bytree": colsample_bytree,
                    "tree_method": "hist",
                    "early_stopping_rounds": early_stopping_rounds,
                }
            elif n_classes > 2:
                kwargs = {
                    "objective": "multi:softprob",
                    "num_class": n_classes,
                    "eval_metric": "mlogloss",
                    "n_estimators": n_estimators,
                    "learning_rate": learning_rate,
                    "max_depth": max_depth,
                    "subsample": subsample,
                    "colsample_bytree": colsample_bytree,
                    "tree_method": "hist",
                    "early_stopping_rounds": early_stopping_rounds,
                }
            else:
                raise ValueError(f"Function validate_model(): Target has no or only 1 class. {n_classes=}")
            m = XGBClassifier(**kwargs, random_state=42 + fold)
            m.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=0)
        elif model == "lgbm":  # LightGBM
            kwargs = {
                "n_estimators": n_estimators,
                "learning_rate": learning_rate,
                "max_depth": max_depth,
                "subsample": subsample,
                "colsample_bytree": colsample_bytree,
                "random_state": 42 + fold,
                "n_jobs": -1,
            }
            m = lightgbm.LGBMClassifier(**kwargs)
            m.fit(X_tr, y_tr, eval_set=[(X_val, y_val)])
        elif model == "catboost":
            kwargs = {
                "iterations": n_estimators,
                "learning_rate": learning_rate,
                "depth": max_depth,
                "random_state": 42 + fold,
                "verbose": False,
            }
            m = CatBoostClassifier(**kwargs)
            m.fit(X_tr, y_tr, eval_set=(X_val, y_val), early_stopping_rounds=early_stopping_rounds, verbose=False)
        elif model == "randomforest":  # very slow, usually bad model
            kwargs = {
                "n_estimators": n_estimators,
                "max_depth": max_depth,
                "max_features": colsample_bytree,  # sklearn accepts float for max_features as fraction of features
                "random_state": 42 + fold,
                "n_jobs": -1,
            }
            m = RandomForestClassifier(**kwargs)
            m.fit(X_tr, y_tr)
        else:
            raise ValueError("Unknown model. Choose from xgb, lgbm, catboost, randomforest.")

        if n_classes == 2:
            oof_preds[val_idx] = m.predict_proba(X_val)[:, 1]
        else:
            oof_preds[val_idx] = m.predict_proba(X_val)
    # set items of oof_preds to 0.499
    # low_thr, high_thr = 0.5, 0.55
    # if n_classes == 2:
    #     mask = (oof_preds > low_thr) & (oof_preds < high_thr)
    #     oof_preds[mask] = 0.499
    # else:
    #     mask = (oof_preds > low_thr) & (oof_preds < high_thr)
    #     oof_preds[mask] = 0.499
    #     renormalize rows to sum to 1 to keep valid probability distributions
        # row_sums = oof_preds.sum(axis=1, keepdims=True)
        # row_sums[row_sums == 0] = 1.0
        # oof_preds = oof_preds / row_sums
    if n_classes == 2:
        auc = roc_auc_score(y, oof_preds)
        logloss = log_loss(y, oof_preds)
        brier = brier_score_loss(y, oof_preds)
    else:
        auc = roc_auc_score(y, oof_preds, multi_class="ovr")
        logloss = log_loss(y, oof_preds)
        brier = np.mean([brier_score_loss((y == c).astype(int), oof_preds[:, c]) for c in range(n_classes)])
    return auc, logloss, brier

#### OOF predictions
Probability predictions for each training sample produced by models that did not train on that sample (via cross‑validation).
They give an approximately unbiased estimate of how the model will behave on new data.
### Metrics (what they measure, range, interpretation)
#### AUC / ROC AUC
What: area under the Receiver Operating Characteristic curve (TPR vs FPR).
Range: 0.0–1.0 (0.5 = random). Higher is better.
Interpretation: measures ranking/separability of positive vs negative; insensitive to predicted‑probability calibration.
#### Log loss (cross‑entropy)
What: average negative log likelihood of the true labels given predicted probabilities.
Range: [0, ∞) (lower is better; 0 is perfect).
Interpretation: penalizes overconfident wrong predictions strongly; sensitive to calibration and probability confidence.
#### Brier score
What: mean squared error between predicted probability and actual outcome (0/1).
Range: [0, 1] for binary targets (lower is better).
Interpretation: measures both calibration and refinement; decomposable into calibration + discrimination components.

### Models
#### LGBM
[Documentation](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html)
[Introductory blogpost](https://www.kdnuggets.com/2023/07/lgbmclassifier-gettingstarted-guide.html)

In [22]:
# COMPARING MODELS PATH
# Next:
#   Die von Kaggle in der Competition benutzte Metric mit aufnehmen
if validate_models:
    # ne_list = [6000]
    ne_list = [2000]
    # lr_list = [0.02]
    lr_list = [0.07]
    # md_list = [6]
    md_list = [4]
    ss_list = [0.8]
    cb_list = [0.2]
    models = ["lgbm"]
    # models = ["xgb", "lgbm", "catboost", "randomforest"]
    print(f"Comparing {len(ne_list)*len(lr_list)*len(md_list)*len(ss_list)*len(cb_list)*len(models)} different models ")
    counter = 0
    tic = time.perf_counter()
    for ne in ne_list:
        for lr in lr_list:
            for md in md_list:
                for ss in ss_list:
                    for cb in cb_list:
                        for model in models:
                            auc, logloss, brier = validate_model(n_estimators=ne, learning_rate=lr, max_depth=md, subsample=ss, colsample_bytree=cb, early_stopping_rounds=50, model=model)
                            new_tic = time.perf_counter()
                            minutes = (new_tic - tic) / 60
                            tic = new_tic
                            counter += 1
                            line = f"{counter};{ne:3};{lr:.3f};{md};{ss:.2f};{cb:.2f};{auc:.6f};{logloss:.6f};{brier:.6f};{model};{minutes:.2f}"
                            print(line)
                            if sound:
                                winsound.Beep(frequency=500, duration=300)
                            # header = "count;ne;lr;md;ss;cb;tm;auc;logloss;brier;model;minutes"
                            with open("model_comparisons.csv", "a", encoding="utf8") as f:
                                f.write(line + "\n")
    if sound:
        winsound.Beep(frequency=1000, duration=1000)


# Submission Path
The following code is only to be executed when generating a file for submission.

In [23]:
# SUBMISSION PATH
if submit_prediction:
    # choose seeds. There will be trained a model for each seed
    # seeds = [42]
    # seeds = [42, 33, 77, 99, 101]
    seeds = [i for i in range(1)]   # change this to run a number of different models

In [24]:
# SUBMISSION PATH
# train models and keep them in a list
if submit_prediction and MODEL == "xgb":
    models = []
    if n_classes == 2:
        kwargs = {
            "objective": "binary:logistic",  # choose objective and eval_metric based on number of classes
            "eval_metric": "logloss",
            "n_estimators": 1,
            "learning_rate": 1,
            "subsample": 0.8,
            "colsample_bytree": 0.2,
            "tree_method": "hist",
            "early_stopping_rounds": 50,
        }
    # elif n_classes > 2:
    #     kwargs = {
    #         "objective": "multi:softprob",
    #         "num_class": n_classes,
    #         "eval_metric": "mlogloss",
    #         "n_estimators": n_estimators,
    #         "learning_rate": learning_rate,
    #         "max_depth": max_depth,
    #         "subsample": subsample,
    #         "colsample_bytree": colsample_bytree,
    #         "tree_method": "hist",
    #         "early_stopping_rounds": early_stopping_rounds,
    #     }
    else:
        raise ValueError(f"Function validate_model(): Target has no or only 1 class. {n_classes=}")
    for s in seeds:
        # take 80% of the trainingsdata, then split these 80% again to obtain trainings and validation data
        X_train, _, y_train, _ = train_test_split(X_training, y, test_size=0.2, random_state=s, stratify=y)
        X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=s, stratify=y_train)

        model = XGBClassifier(**kwargs, early_stopping_rounds=50, random_state=s)
        model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=0)
        models.append(model)

In [25]:
# SUBMISSION PATH
# train models and keep them in a list
import lightgbm as lgb
from sklearn.model_selection import train_test_split

if submit_prediction and MODEL == "lgbm":
    models = []
    kwargs = {
        "n_estimators": 5000,
        "learning_rate": 0.05,
        "max_depth": -1,          # often better than forcing 6
        "subsample": 0.8,
        "colsample_bytree": 0.8,  # 0.2 can be too restrictive
        "random_state": 42,
        "n_jobs": -1,
        "objective": "regression",
    }

    for s in seeds:
        X_train, _, y_train, _ = train_test_split(
            X_training, y, test_size=0.2, random_state=s
        )
        X_tr, X_val, y_tr, y_val = train_test_split(
            X_train, y_train, test_size=0.2, random_state=s
        )

        suppress_python_output()
        model = lgb.LGBMRegressor(**kwargs)

        model.fit(
            X_tr, y_tr,
            eval_set=[(X_val, y_val)],
            eval_metric="rmse",
            callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=False)]
        )
        restore_python_output()

        models.append(model)

In [26]:
# SUBMISSION PATH
# Make sure column order in test data matches training data:
if submit_prediction:
    X_test_final = X_test[X_training.columns]
X_test_final

KeyError: "['exam_score'] not in index"

In [26]:
# SUBMISSION PATH
# predict on final test set and average probs
if submit_prediction:
    probs = []
    for model in models:
        bi = getattr(model, "best_iteration", None)
        if bi is not None:
            probs.append(model.predict_proba(X_test_final, iteration_range=(0, bi + 1))[:, 1])
        else:
            probs.append(model.predict_proba(X_test_final)[:, 1])
        # probs shape: seeds x samples x classes  , classes is = 1 here due to indexing

    probs_arr = np.vstack(probs)   # shape: (n_models, n_samples)
    mean_prob = probs_arr.mean(axis=0)

    # set items of mean_prob to 0.0 if they are < 0.2 and to 1.0 if they are > 0.8
    # mean_prob = np.asarray(mean_prob, dtype=float)  # ensure numpy array
    # low_thr = 0.5
    # high_thr = 0.5
    # mean_prob[mean_prob < low_thr] = 0.0
    # mean_prob[mean_prob > high_thr] = 1.0
# mean_prob

ValueError: need at least one array to concatenate

In [ ]:
# SUBMISSION PATH (REGRESSION)
if submit_prediction:
    preds = []
    for model in models:
        bi = getattr(model, "best_iteration_", None)  # note the trailing underscore in sklearn API
        if bi is not None:
            preds.append(model.predict(X_test_final, num_iteration=bi))
        else:
            preds.append(model.predict(X_test_final))

    preds_arr = np.vstack(preds)      # (n_models, n_samples)
    mean_pred = preds_arr.mean(axis=0)

# mean_pred is your final prediction vector

In [ ]:
"""
2) 0.70160
OneHot: ["ethnicity", "employment_status"]
Mapping: ["gender", "education_level", "income_level", "smoking_status"]
Fitting: 80% from 80%, 10 fits, "n_estimators": 2000
3) 0.70159
Fitting: 80% from 80%, 10 fits, "n_estimators": 2000
4) 0.70163
Fitting: 80% from 80%, 10 fits, "n_estimators": 3000
5) debugging
6) 0.70101
Fitting: 80% from 80%, 2 fits, "n_estimators": 5000
7) 0.70166
Fitting: 80% from 80%, 15 fits, "n_estimators" etc. 5000, 0.03, 4, 0.8, 0.8
8) 0.70191
Fitting: 80% from 80%, 15 fits, "n_estimators" etc. 8000, 0.01, 5, 0.9, 0.5
9) 0.70301
Fitting: LGBM!!! 80% from 80%, 15 fits, "n_estimators" etc. 2000, 0.07, 4, 0.8, 0.8
10) 0.70084
Fitting: LGBM!!! 80% from 80%, 15 fits, "n_estimators" etc. 2000, 0.07, 4, 0.8, 0.8
<0.2 -> 0.0, >0.8 -> 1.0
11) 0.70300
Fitting: LGBM!!! 80% from 80%, 15 fits, "n_estimators" etc. 2000, 0.07, 4, 0.8, 0.8
<0.05 -> 0.0, >0.95 -> 1.0
12) 0.61863
Fitting: LGBM!!! 80% from 80%, 15 fits, "n_estimators" etc. 2000, 0.07, 4, 0.8, 0.8
<0.5 -> 0.0, >0.5 -> 1.0
13) 0.70324
Fitting: LGBM!!! 80% from 80%, 15 fits, "n_estimators" etc. 4000, 0.03, 4, 0.9, 0.5
14) 0.70364
Fitting: LGBM, 80% from 80%, 15 fits, "n_estimators" etc. 6000, 0.02, 6, 0.8, 0.2
15) 0.70222
NO ORIGINAL DATA!!!!, Fitting: LGBM, 80% from 80%, 15 fits, "n_estimators" etc. 6000, 0.02, 6, 0.8, 0.2
16) 0.70281
+6 derived features TG_HDL_Ratio LDL_HDL_Ratio MAP Pulse_Pressure Central_Obesity_Index Sleep_Deviation
Fitting: LGBM, 80% from 80%, 15 fits, "n_estimators" etc. 6000, 0.02, 6, 0.8, 0.2
17) 0.70322
+1 physical_inactivity
Fitting: LGBM, 80% from 80%, 15 fits, "n_estimators" etc. 40000, 0.005, 6, 0.8, 0.2
18) 0.70381
+1 physical_inactivity
Fitting: LGBM, 80% from 80%, 15 fits, "n_estimators" etc. 6000, 0.02, 6, 0.8, 0.2
19) 0.70341
+1 physical_inactivity -1 physical_activity_minutes_per_week
Fitting: LGBM, 80% from 80%, 15 fits, "n_estimators" etc. 6000, 0.02, 6, 0.8, 0.2
20) 0.70376
+1 physical_inactivity, Replaced age with age_squared, +1 triglycerides_root
Fitting: LGBM, 80% from 80%, 15 fits, "n_estimators" etc. 6000, 0.02, 6, 0.8, 0.2
21) 0.70384
+1 physical_inactivity, Replaced age with age_squared
Fitting: LGBM, 80% from 80%, 15 fits, "n_estimators" etc. 6000, 0.02, 6, 0.8, 0.2
"""
pass

### Learnings
#### Model
- Random Forest is slow and bad.
- LGBM ist faster and better than XGBoost.
- Parameter "subsample" apparently has no effect on LGBM.
- The higher n_estimators, the lower is the optimal learning rate.
- 40000 perform better than 6000 in training data but worse with test data.
#### Feature Enhancement
- Setting the ID of all trainings data to zero worsens the outcome.
- Ordinal Mapping instead of One HoT Encoding: no effect on XGB, bad effect on LGBM.
- Tried 12 derived features. All worsened the outcome. Replacing base features with the features derived from them also worsened the outcome.
- Removing any base feature worsens the score. The by far most important features are physical activity, diabetes family history, age, tryglycerides.
#### Data
- Including the original data for training worsens the validation outcome but improves the kaggle result on test data.
- Including the original several times (as duplicates) has no effect.